In [32]:
import scipdf
import sys
sys.path.append('../')

from pdf_parser import parse_pdf_content, parse_pdf_abstract, generate_user_input 

This is the system prompt for generating the reviews.

In [33]:
SYSTEM_PROMPT = """
You are a professional machine learning conference reviewer who reviews a given paper and considers 4 criteria: [Significance and novelty], [Potential reasons for acceptance], [Potential reasons for rejection], and [Suggestions for improvement]. Please ensure that for each criterion, you summarize and provide random number of detailed supporting points from the content of the paper. And for each supporting point within each of criteria, use the format: '<title of supporting point>' followed by a detailed explanation. The criteria you need to focus on are:

1. [Significance and novelty]: Assess the importance of the paper in its research field and the innovation of its methods or findings。
2. [Potential reasons for acceptance]: Summarize reasons that may support the acceptance of the paper, based on its quality, research results, experimental design, etc.
3. [Potential reasons for rejection]: Identify and explain flaws or shortcomings that could lead to the paper's rejection.
4. [Suggestions for improvement]: Provide specific suggestions to help the authors improve the paper and increase its chances of acceptance.

After reading the content of the paper provided below, your response should only include your reviews only, which means always start with [Significance and novelty], dont' repeat the given paper and output things other than your reviews in required format, just extract and summarize information related to these criteria from the provided paper. The paper is given as follows:
"""

Parse the PDF file and extract the text.

In [28]:
pdf = scipdf.parse_pdf_to_dict('../demo/Transformers.pdf')

# Model Inference

In [34]:
import torch
from transformers import AutoTokenizer
from peft import AutoPeftModelForCausalLM

In [ ]:
model_id = "travis0103/mistral_7b_paper_review_lora"

model = AutoPeftModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
)

tokenizer = AutoTokenizer.from_pretrained(model_id)

In [ ]:
abstract = parse_pdf_abstract(pdf) 
abstract_input = generate_user_input(abstract) 
print(abstract_input)

In [49]:
messages = [
    {"role": "user", "content": SYSTEM_PROMPT + "\n\n" + abstract_input},
]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoded_input = tokenizer.apply_chat_template(messages, return_tensors="pt").to(device)
generated_ids = model.generate(encoded_input, max_new_tokens=1024, do_sample=True, pad_token_id=tokenizer.eos_token_id, eos_token_id=tokenizer.eos_token_id)
decoded_output = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

In [44]:
import re

def extract_sections(text):
    # Define the pattern to capture each section
    pattern = r"\[(Significance and novelty|Potential reasons for acceptance|Potential reasons for rejection|Suggestions for improvement)\]" \
            r"(.*?)(?=\n\[(Significance and novelty|Potential reasons for acceptance|Potential reasons for rejection|Suggestions for improvement)\]|\Z)"
    
    # Use re.findall to get all matches
    sections = re.findall(pattern, text, flags=re.DOTALL)

    # Convert list of tuples into a dictionary for easier access
    section_dict = {section[0]: section[1].strip() for section in sections}
    
    final_output = ""
    for key, value in section_dict.items():
        final_output += f"[{key}]\n{value}\n\n"
        
    return final_output


In [45]:
print(extract_sections(decoded_output))


[Significance and novelty]
<Innovation in model architecture> The paper introduces a new and innovative way of doing sequence-to-sequence translation with the transformer architecture, demonstrating significant advantages over previous methods through extensive experimental evaluation.
<Thorough experimental evaluation> The paper conducts a thorough evaluation and comparison against other attention models, addressing concerns and demonstrating the superiority of the proposed approach through substantial improvements in translation performance.

[Potential reasons for acceptance]
<Impressive experimental results> The proposed transformer architecture achieves strong results and advances the state-of-the-art in sequence-to-sequence translation tasks.
<Addressing concerns> The authors have made effort to address the concerns raised by the reviewers and have committed to improving the paper based on the feedback received.

[Potential reasons for rejection]
<Inadequate experimental evaluati

## Run Inference on review results

In [ ]:
import pandas as pd 

# GPT Inference

In [15]:
import os
import openai

os.environ["OPENAI_API_KEY"] = ""
client = openai.Client()

In [ ]:
full_content = parse_pdf_content(pdf)
full_input = generate_user_input(full_content)

In [16]:
response = client.chat.completions.create(
    model="gpt-4-turbo",
    messages=[
      {"role": "system", "content": SYSTEM_PROMPT},
      {"role": "user", "content": full_input},
    ]
)

In [17]:
print(response.choices[0].message.content)

[Significance and novelty]
The "Transformer" paper introduces a novel architecture that challenges the dominant sequence transduction models which rely on recurrent neural networks (RNNs) and convolutional neural networks (CNNs). The main contributions and innovations of this work include the introduction of an architecture based entirely on attention mechanisms, eliminating the need for recurrence or convolution within the model, which represents a significant shift in the approach to machine translation and potentially other sequence modeling tasks.

- 'Elimination of recurrence and convolution': The proposed Transformer model is novel in its exclusive reliance on attention mechanisms, avoiding traditional RNNs and CNNs which can be computationally expensive and slow due to their inherent sequential nature.
- 'Optimization for Parallelization': The Transformer is designed to maximize parallel computational capabilities, reducing training time significantly compared to existing models